In [2]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import datetime
import warnings
warnings.filterwarnings('ignore')

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff

py.offline.init_notebook_mode(connected=True)

from math import floor
from plotly import tools
from plotly.graph_objs import *

In [3]:
data = pd.read_csv("../data/cleaned_data.csv")

In [4]:
data["profit"] = data["revenue"] - data["budget"]

In [5]:
data["profit"].describe()

count    4.790000e+03
mean     5.336400e+07
std      1.361221e+08
min     -1.657101e+08
25%     -7.985852e+05
50%      2.602056e+06
75%      5.546026e+07
max      2.550965e+09
Name: profit, dtype: float64

In [6]:
data[data["profit"]==data["profit"].max()]

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,vote_average,vote_count,title,cast,crew,year,month,day,dow,profit
0,237000000,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['culture clash', 'future', 'space war', 'spac...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"['Ingenious Film Partners', 'Twentieth Century...","['United States of America', 'United Kingdom']",2009-12-10,2787965087,...,7.2,11800,Avatar,Sam Worthington,"[{'credit_id': '52fe48009251416c750aca23', 'de...",2009,12,10,3,2550965087


In [28]:
profit_by_dow = data[data["year"]>1990].groupby(["dow"]).profit.mean().reset_index()

In [29]:
bar_data = [go.Bar(x=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']\
                   , y=profit_by_dow["profit"])]

py.offline.iplot({ 'data': bar_data,
            'layout': {
               'title': 'Average Profit by Day of Week',
               'xaxis': {
                 'title': 'Day of Week'},
               'yaxis': {
                'title': 'Profit'}
        }})

#### Release movies on Tuesday 

### Violin Plots

In [9]:
def extract_decade(x):
    return str(floor(x/10)*10)+"s"

In [10]:
data["decade"] = data["year"].apply(extract_decade)

In [11]:
df = data

In [12]:
df = df.sort_values(by=['decade'], ascending=True)

In [13]:
df.groupby(['decade']).size()

decade
1910s       1
1920s       4
1930s      15
1940s      25
1950s      27
1960s      71
1970s     109
1980s     278
1990s     778
2000s    2044
2010s    1438
dtype: int64

In [14]:
df[df['decade'] == '1910s']

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,vote_count,title,cast,crew,year,month,day,dow,profit,decade
4581,385907,['Drama'],"['usa', 'naivety', 'intolerance', 'mill', 'mar...",en,"The story of a poor young woman, separated by ...",3.232447,"['Triangle Film Corporation', 'Wark Producing ...",['United States of America'],1916-09-04,8394751,...,60,Intolerance,Lillian Gish,"[{'credit_id': '577fe3e79251415db5000bf2', 'de...",1916,9,4,0,8008844,1910s


### get rid of the 1910's decade since it has only 1 movie

In [15]:
df = df[df['decade'] != '1910s']

In [16]:
da = []
for i in range(0,len(pd.unique(df['decade']))):
    trace = {
            "type": 'violin',
            "x": df['decade'][df['decade'] == pd.unique(df['decade'])[i]],
            "y": df['vote_average'][df['decade'] == pd.unique(df['decade'])[i]],
            "name": pd.unique(df['decade'])[i],
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        }
    da.append(trace)

        
fig = {
    "data": da,
    "layout" : {
        "title": "Average Movie Ratings by Decade",
            "xaxis" : dict(title = 'Decade', autotick=False, showticklabels=True),
            "yaxis" : dict(title = 'Average Rating')
    }
}

iplot(fig, validate = False)

#### movies were better in the 1990's than the 2010's

In [16]:
df.columns

Index(['budget', 'genres', 'keywords', 'original_language', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages',
       'vote_average', 'vote_count', 'title', 'cast', 'crew', 'year', 'month',
       'day', 'dow', 'profit', 'decade'],
      dtype='object')

### Bubble Chart

In [17]:
layout = go.Layout(
    title='Profit vs. Budget',
    xaxis=dict(
        title='Budget',
        gridcolor='rgb(255, 255, 255)',
        range=[0, 4e8],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Profit',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    )
)
data = [go.Scatter(
    x=df.budget.values, # Budget
    y=df.profit.values,  # Gross
    mode='markers',
    text= df.title.values, # Movie Titles
    marker=dict(
        size=3*(df.vote_average),
        sizeref=1.0,
        color=df.vote_average.values,
        colorbar=ColorBar(title='Average Rating'),
        showscale=True,
        colorscale='Viridis'
    ))]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Runtime vs Profit

### Actors vs Profit

### Directors vs Profit

### Month vs Profit

In [18]:
df.columns

Index(['budget', 'genres', 'keywords', 'original_language', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages',
       'vote_average', 'vote_count', 'title', 'cast', 'crew', 'year', 'month',
       'day', 'dow', 'profit', 'decade'],
      dtype='object')

In [19]:
df.head().transpose()

,2638,4650,4448,4583,3802
budget,92620000,245000,0,379000,3950000
genres,"['Drama', 'Science Fiction']","['Drama', 'Romance', 'War']","['Drama', 'Thriller', 'Romance']","['Drama', 'Music', 'Romance']","['Action', 'Drama', 'History']"
keywords,"['man vs machine', 'underground world', 'inven...","['world war i', 'silent film']","['london england', 'casino', 'irony', 'forbidd...","['musical', 'singer', 'pre-code', 'wisecrack h...","['world war i', 'zeppelin', 'royal air force',..."
original_language,de,en,de,en,en
overview,In a futuristic city sharply divided between t...,The story of an idle rich boy who joins the US...,The rise and inevitable fall of an amoral but ...,"Harriet and Queenie Mahoney, a vaudeville act,...",Two brothers attending Oxford enlist with the ...
popularity,32.3515,0.785744,1.82418,0.968865,8.48412
production_companies,"['Paramount Pictures', 'Universum Film (UFA)']",['Metro-Goldwyn-Mayer (MGM)'],['Nero Films'],['Metro-Goldwyn-Mayer (MGM)'],['The Caddo Company']
production_countries,['Germany'],['United States of America'],['Germany'],['United States of America'],['United States of America']
release_date,1927-01-10,1925-11-05,1929-01-30,1929-02-08,1930-11-15
revenue,650422,22000000,0,4358000,8000000


### Runtime vs Profit

### Remove movies with a runtime of 0

In [20]:
df2 = df[df.runtime > 20]
df2 = df2[df.vote_average>3]

/Users/mbk/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [21]:
layout = go.Layout(
    title='Profit vs. Runtime',
    xaxis=dict(
        title='Runtime',
        gridcolor='rgb(255, 255, 255)',
        range=[50, 250],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Profit',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    )
)
data = [go.Scatter(
    x=df2.runtime.values, # Budget
    y=df2.profit.values,  # Gross
    mode='markers',
    text= df2.title.values, # Movie Titles
    marker=dict(
        size=3*(df2.vote_average),
        sizeref=1.0,
        color=df2.vote_average.values,
        colorbar=ColorBar(title='Average Rating'),
        showscale=True,
        colorscale='Viridis'
    ))]
fig = go.Figure(data=data, layout=layout)
iplot(fig)